In [1]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
import operator
import math
from numpy import unravel_index



UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, 
                 smoothing_obs = 0.01, 
                 transition_proba1= None,
                 prob_abs= 0.00001):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij,a_ik = Pr(Y_(t+1)=q_i|Y_t=q_j,Y_(t-1)=q_k)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            self.prob_abs = prob_abs
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba1 is None:
                self.transition_proba1 = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba1=transition_proba1
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        
        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j,Y_(t-1)=q_k)
            """
            # fill with counts
            for triple in trans_counts:
                i=self.Y_index[triple[2]]
                j=self.Y_index[triple[1]]
                k=self.Y_index[triple[0]]
                self.transition_proba[k,j,i]=trans_counts[triple]
            # normalize sorun cıkacak !!!
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(self.N,self.N)
            
                
        def transition_estimation1(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba1[j,i]=trans_counts[pair]
            # normalize
            self.transition_proba1=self.transition_proba1/self.transition_proba1.sum(axis=0).reshape(1,self.N)
        
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts,trans_counts1):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.transition_estimation1(trans_counts1)
            self.init_estimation(init_counts)
        
        def viterbi(self,observations):
            if len(observations)<2:
                return [hmm.Y_index[z] for z in observations]
            nSamples = len(observations)
            nStates = self.transition_proba.shape[0] # number of states
            c = np.zeros(nSamples) #scale factors (necessary to prevent underflow)
            viterbi = np.zeros((nStates,nStates,nSamples)) # initialise viterbi table
            viterbi1 = np.zeros((nStates,nSamples)) # initialise viterbi table
            psi = np.zeros((nStates,nStates,nSamples)) # initialise the best path table
            best_path = np.zeros(nSamples); # this will be your output
            
            idx0 = self.X_index[observations[0]]
            idx1 = self.X_index[observations[1]]

            viterbi1[:,0] = self.initial_state_proba.T * self.observation_proba[idx0,:].T

            for s in range (0,nStates): # loop through the states @(t-2)
                for v in range (0,nStates): # loop through the states @(t-1)
                    viterbi[s,v,1] = viterbi1[s,0] * self.transition_proba1[s,v] * self.observation_proba[idx1,v]

            
       
            psi[0] = 0;

            for t in range(2,nSamples): # loop through time
                idx = self.X_index[observations[t]]
                for s in range (0,nStates): # loop through the states @(t-1)
                    for v in range (0,nStates): # loop through the states @(t-1)
                        self.transition_proba[np.isnan(self.transition_proba)] = self.prob_abs
                        trans_p = viterbi[:,s,t-1] * self.transition_proba[:,s,v]
                
                        if(math.isnan(trans_p[0])):
                            trans_p[0]=0

                        psi[s,v,t], viterbi[s,v,t] = max(enumerate(trans_p), key=operator.itemgetter(1))
                        viterbi[s,v,t] = viterbi[s,v,t]*self.observation_proba[idx,v]


            cabbar = viterbi[:,:,nSamples-1]
            best_path[nSamples-1] = unravel_index(cabbar.argmax(), cabbar.shape)[1]
            best_path[nSamples-2] = unravel_index(cabbar.argmax(), cabbar.shape)[0]

            
            for t in range(nSamples-3,-1,-1): # states of (last-1)th to 0th time step
                best_path[t] = psi[best_path[t+1],best_path[t+2],t+2]

            return best_path

# Compter les mots et les tags

In [2]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    c_transitions1 = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            # j never is seen at second position
            if i >= 1:
                trans1 = (sent[i-1][1],tag)
                if trans1 in c_transitions1:
                    c_transitions1[trans1]=c_transitions1[trans1]+1
                else:
                    c_transitions1[trans1]=1
                
            if i > 1:
                trans = (sent[i-2][1],sent[i-1][1],tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits, c_transitions1


# Création du vocabulaire (filtrage selon le nombre d'occurence)

In [3]:
def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    voc.append(UNK)
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


# les données


In [4]:
import pickle

with open('typos-data/train10.pkl', 'rb') as f:
    train10 = pickle.load(f)
with open('typos-data/test10.pkl', 'rb') as f:
    test10 = pickle.load(f)
with open('typos-data/train20.pkl', 'rb') as f:
    train20 = pickle.load(f)
with open('typos-data/test20.pkl', 'rb') as f:
    test20 = pickle.load(f)

In [5]:
print "Nombre de lettres de train10 = "+str(len(train10))
print "Nombre de lettres de test10  = "+str(len(test10))
print "Nombre de lettres de train20 = "+str(len(train20))
print "Nombre de lettres de test20  = "+str(len(test20))


Nombre de lettres de train10 = 29057
Nombre de lettres de test10  = 1501
Nombre de lettres de train20 = 27184
Nombre de lettres de test20  = 3374


In [6]:
cwords,ctags,cpairs,ctrans,cinits,ctrans1 = make_counts(train10)
print "Nombre de lettre  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))

vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de lettre  : 26
Nombre de tags  : 26
Nombre de paires: 127
Nombre de trans : 2489 / 144
Nombre de init. : 26
Vocabulaire :27


# Apprentissage en une fois

In [9]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 0.4,
                 prob_abs= 0)
hmm.supervised_training(cpairs,ctrans,cinits,ctrans1)

HMM creating with: 
26 states
27 observations


/Users/macbook975/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:135: RuntimeWarning: invalid value encountered in divide


In [10]:
err = 0
tot = 0
for i in range(len(test10)):
    b = [z[0] for z in test10[i]]
    out = hmm.viterbi(b)
    bb = [z[1] for z in test10[i]]
    cc = [hmm.Y_index[z] for z in bb]
    #print(out,cc,b)
    err += len([x for x in out if x in cc])
    tot += len(out)
print(err)
print(tot)
float(err)/tot

/Users/macbook975/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


6964
7320


0.9513661202185792